In [75]:
import torch
import numpy as np
from idem_net_mnist import IdemNetMnist
from data_loader import load_MNIST
import matplotlib.pyplot as plt

In [76]:
run_id = "mnist20241121-142447"
epoch_num = "final.pth"

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')


checkpoint_path = f"checkpoints/{run_id}/{epoch_num}"
device

device(type='mps')

In [77]:
model = IdemNetMnist()

state_dict = torch.load(checkpoint_path, weights_only=True, map_location=device)
# state_dict = state_dict["model_state_dict"]

In [78]:
model.load_state_dict(state_dict)
#model.eval()

<All keys matched successfully>

In [79]:
def plot_generation(inputs, outputs, outputs2, num_images=5, show=False):
    """
    Plots input and output image pairs side by side.
    
    Parameters:
    - inputs: Batch of input images, expected shape (batch_size, height, width).
    - outputs: Batch of output images, expected shape (num_applications, batch_size, height, width).
    - num_images: Number of image pairs to display (default is 5).
    """
    # Limit the number of images to the smaller of num_images or batch size
    num_images = min(num_images, len(inputs), len(outputs))
    
    plt.figure(figsize=(8, num_images * 3))
    for i in range(num_images):
        # Plot input image
        plt.subplot(num_images, 3, 2 * i + 1)
        plt.imshow(inputs[i].squeeze().detach().numpy(), cmap='gray')
        plt.title("Input")
        plt.axis('off')
        
        # Plot output image
        plt.subplot(num_images, 3, 2 * i + 2)
        plt.imshow(outputs[i].squeeze().detach().numpy(), cmap='gray')
        plt.title("Output")
        plt.axis('off')

        # Plot output image
        plt.subplot(num_images, 3, 2 * i + 3)
        plt.imshow(outputs2[i].squeeze().detach().numpy(), cmap='gray')
        plt.title("Output")
        plt.axis('off')

    plt.tight_layout()
    if show:
        plt.show()



In [80]:
# Generate random images based on noise with the same shape as the input images
num_noise = 2
noise = torch.randn(num_noise, 1, 28, 28)

# Do interpolation between the first two images
num_interpolations = 30
interpolations = torch.zeros(num_interpolations, 1, 1, 28, 28)
outputs = torch.zeros(num_interpolations, 1, 1, 28, 28)
outputs2 = torch.zeros(num_interpolations, 1, 1, 28, 28)
for i in range(num_interpolations):
    alpha = i / (num_interpolations - 1)
    interpolations[i] = alpha * noise[0] + (1 - alpha) * noise[1]





In [81]:
import os

output_dir = "output_images_latentspace"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# output for interpolations
for i in range(num_interpolations):
    outputs[i] = model(interpolations[i])
    outputs2[i] = model(outputs[i])
    plot_generation(interpolations[i], outputs[i], outputs2[i], num_images=1)
    # Save plot
    plt.savefig(os.path.join(output_dir, f"interpolation_{i}.png"))
    plt.close()
